### Ejercicio 18: Aceite

Una empresa elabora un aceite nuevo (REFI) refinando exclusivamente diferentes tipos de
aceite y mezclándolos. Los tipos de aceite se clasifican en dos categorías: vegetales (VEG1 y
VEG2) y no vegetales (OIL1, OIL2 y OIL3). Dependiendo del tipo de aceite, vegetal o no vegetal,
se requiere una línea de producción distinta para refinarlo, por lo que se puede refinar un máximo
de 200 toneladas de aceite vegetal y 250 de no vegetal. Además, se puede asumir que el coste del
refinamiento es nulo y que durante este proceso no se producen pérdidas de peso. Por otro lado,
existen restricciones de control de calidad que imponen cotas (inferior y superior) a la acidez
del producto final: 1.4 y 1.8 unidades, respectivamente. Se puede asumir que la acidez se mezcla
linealmente. La acidez y el coste de una tonelada de cada tipo de aceite se re
eja en la siguiente
tabla:
 Aceite | VEG1 | VEG2 | OIL1 | OIL2 | OIL3 
-----------------|---------------|---------------|---------------|---------------|----------------------------
 Coste  | 110           | 120           | 130           | 110           | 115         
 Acidez | 2,2           | 1,1           | 2             | 1,2           | 1,5 

Cada tonelada de producto final se vende a un precio de 150 euros. Plantea el modelo que
maximiza el beneficio neto.

$$
\begin{equation*}
    \begin{aligned}&\text{min}&& z=150\sum_v F_v+150\sum_o F_o-\sum_v coste_v F_v-\sum_o coste_o F_o \\
    & \text{sujeto a:} \\
    &  && \sum_v acidez_v F_v+\sum_o acidez_o F_o \leq 1.8 (\sum_v F_v+\sum_o F_o)  && \\
    &  && \sum_v acidez_v F_v+\sum_o acidez_o F_o \geq 1.4 (\sum_v F_v+\sum_o F_o)  && \\
    &  && F_v \in (VEG1, VEG2), F_o \in (OIL1, OIL2, OIL3) &&\\
    &  && F_v, F_o \geq 0 
    \end{aligned}
    \end{equation*}
$$

In [1]:
from pyomo.environ import *
import numpy as np
model = ConcreteModel()
veg = np.arange(2)
oil = np.arange(2,5)
coste= [110,120,130,110,115]
acidez= [2.2,1.1,2,1.2,1.5]

model.v = Set(initialize=veg)
model.o = Set(initialize=oil)

In [2]:
model.Fv = Var(model.v, domain = NonNegativeReals)
model.Fo = Var(model.o, domain = NonNegativeReals)
model.pprint()

2 Set Declarations
    o : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {2, 3, 4}
    v : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {0, 1}

2 Var Declarations
    Fo : Size=3, Index=o
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
    Fv : Size=2, Index=v
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals

4 Declarations: v o Fv Fo


In [3]:
model.benef = Objective(expr = 150*(sum(model.Fv[i] for i in model.v)+sum(model.Fo[j] for j in model.o))-sum(model.Fv[i]*coste[i] for i in model.v)-sum(model.Fo[j]*coste[j] for j in model.o), sense = maximize)
model.benef.pprint()

benef : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 150*(Fv[0] + Fv[1] + Fo[2] + Fo[3] + Fo[4]) - (110*Fv[0] + 120*Fv[1]) - (130*Fo[2] + 110*Fo[3] + 115*Fo[4])


In [4]:
model.maxprodv = Constraint(expr = sum(model.Fv[i] for i in model.v) <= 200)
model.maxprodo = Constraint(expr = sum(model.Fo[i] for i in model.o) <= 250)
model.acup = Constraint(expr= sum(model.Fv[i]*acidez[i] for i in model.v)+sum(model.Fo[i]*acidez[i] for i in model.o) <=1.8*(sum(model.Fv[i] for i in model.v)+sum(model.Fo[j] for j in model.o)) )
model.aclo = Constraint(expr= sum(model.Fv[i]*acidez[i] for i in model.v)+sum(model.Fo[i]*acidez[i] for i in model.o) >=1.4*(sum(model.Fv[i] for i in model.v)+sum(model.Fo[j] for j in model.o)) )

model.acup.pprint()

acup : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                                                                  : Upper : Active
    None :  -Inf : 2.2*Fv[0] + 1.1*Fv[1] + 2*Fo[2] + 1.2*Fo[3] + 1.5*Fo[4] - 1.8*(Fv[0] + Fv[1] + Fo[2] + Fo[3] + Fo[4]) :   0.0 :   True


In [5]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()   

2 Set Declarations
    o : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {2, 3, 4}
    v : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {0, 1}

2 Var Declarations
    Fo : Size=3, Index=o
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
          3 :     0 : 250.0 :  None : False : False : NonNegativeReals
          4 :     0 :   0.0 :  None : False : False : NonNegativeReals
    Fv : Size=2, Index=v
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 : 200.0 :  None : False : False : NonNegativeReals
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    benef : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 15